# missing data interpolation

statistics is the answer to everything

### potential shenanigans

"Several techniques have been used to fill the gaps in either the UWLS or OI derived total vector maps.

These are implemented using covariance derived from normal mode analysis (Lipphardt et al. 2000), open-boundary modal analysis (OMA) (Kaplan and Lekien 2007), and empirical orthogonal function (EOF) analysis (Beckers and Rixen 2003; Alvera-Azcárate et al. 2005); and using idealized or smoothed observed covariance (Davis 1985)."

- normal mode analysis
- open-boundary modal analysis (OMA)
- empirical orthogonal function analysis (EOF)
- use idealized/smoothed observed covariance

---

### other ideas

DINEOF (could only find an implementation in R)

to be honest I don't understand any of these methods but they look cool

### currently implemented:

rip data straight from the lower resolution data for areas where data is considered missing in the high resolution data

In [ ]:
%matplotlib inline

In [ ]:
import math
import time
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
from parcels import FieldSet
from datetime import timedelta, datetime

In [ ]:
DATA_6KM = 6
DATA_2KM = 2
DATA_1KM = 1

filename_dict = {
    DATA_6KM: "west_coast_6km_hourly",
    DATA_2KM: "west_coast_2km_hourly",
    DATA_1KM: "west_coast_1km_hourly"
}

def get_file_info(path, res):
    xrds = xr.open_dataset(path)
    # flat mesh so the velocity data isn't converted when retrieving interpolated velocities
    fs = FieldSet.from_xarray_dataset(
        xrds.copy(deep=True),
        dict(U="u",V="v"),
        dict(lat="lat",lon="lon",time="time"),
        mesh="flat"
    )
    xrds.close()
    lat = xrds["lat"].values
    lon = xrds["lon"].values
    return dict(
        path = path,
        res = res,
        xrds = xrds,
        fs = fs,
        timerng = (xrds["time"].min().values, xrds["time"].max().values),
        lat = lat,
        lon = lon,
        domain = {
            "S": lat.min(),
            "N": lat.max(),
            "W": lon.min(),
            "E": lon.max(),
        }
    )

### target and interp_references

`target` is the data you are interpolating.

`interp_references` is a list of reference data to interpolate from. A few specifications:
- should be ordered from most accurate data to least accurate
- time domain should be identical or bigger than the one of the target
- lat and lon domain should be bigger than the target's to prevent any out-of-bounds complications

In [ ]:
target = get_file_info("west_coast_1km_hourly_netcdfs/west_coast_1km_hourly_region_tijuana_river.nc", DATA_1KM)

interp_references = [
    get_file_info("west_coast_2km_hourly_netcdfs/west_coast_2km_hourly_region0.nc", DATA_2KM),
    get_file_info("west_coast_6km_hourly_netcdfs/west_coast_6km_hourly_region0.nc", DATA_6KM)
]

### interpolation type

more information can be found in the `tutorial_interpolation` notebook

it's mostly finding whether `linear` or `linear_invdist_land_tracer` works better

FUN FACT: parcels lets you change the interp method for U and V fields to `linear_invdist_land_tracer` and even access values from it, but doesn't work when you stick the FieldSet into a ParticleSet.

maybe this means `linear_invdist_land_tracer` shouldn't be used for velocity interpolation?

In [ ]:
reference_interp_method = "linear"
# reference_interp_method = "linear_invdist_land_tracer"

for r in interp_references:
    r["fs"].U.interp_method = reference_interp_method
    r["fs"].V.interp_method = reference_interp_method

## nan values and parcels

note that when this xarray Dataset is passed into parcels, all the nan values change to 0 and the mask generation won't work anymore

so the Dataset is copied for use with the FieldSet instead

In [ ]:
def generate_mask(data):
    """
    Generates a boolean mask signifying which points in the data are invalid.
    
    Args:
        data (np.ndarray): an array with the shape of (time, lat, lon)
    """
    mask = np.zeros(data.shape, dtype=bool)
    for i in range(data.shape[1]):
        for j in range(data.shape[2]):
            point = data.T[j][i]
            nan_vals = np.isnan(point)
            # if the point at (lat, lon) contains real data and nan values
            # mark those points as invalid
            if not nan_vals.all():
                mask.T[j][i][:] = np.where(nan_vals.flatten(), 1, 0)
    return mask

In [ ]:
invalid = generate_mask(target["xrds"]["u"].values)
num_invalid = invalid.sum()
print(f"total invalid values on target data: {num_invalid}")

In [ ]:
invalid_pos = np.where(invalid)
invalid_pos

### use of Parcels Field for interpolation

indexing Field values goes [time, depth, lat, lon]

Field does interpolation automatically when indexing values between it's coordinate values

### note

https://stackoverflow.com/questions/12923586/nearest-neighbor-search-python

in theory, the latitude and longitude values are equally spaced. however, the difference between the coordinate values always flucuates a very small amount between two distinct values, so it's not perfectly equally spaced.

from testing, this causes enough error to completely change a simulation, so a kdtree must be used.

In [ ]:
import scipy.spatial

for f in interp_references:
    f["latkdtree"] = scipy.spatial.cKDTree(np.array([f["lat"]]).T)
    f["lonkdtree"] = scipy.spatial.cKDTree(np.array([f["lon"]]).T)

In [ ]:
def get_interped(i, ref, vec, invalid_where):
    """
    Args:
        i (int)
        ref (dict)
        vec (str): either u or v
        invalid_where (array-like)
    
    Returns:
        float: nan if no data was found, interpolated value otherwise
    """
    time_diff = np.diff(ref["fs"].U.grid.time)[0]
    t = invalid_where[0][i]
    lat = target["lat"][invalid_where[1][i]]
    lon = target["lon"][invalid_where[2][i]] 
    if np.isnan(ref["xrds"][vec].values[t, ref["latkdtree"].query([lat])[1], ref["lonkdtree"].query([lon])[1]]):
        return np.nan
    if vec == "u":
        return ref["fs"].U[t * time_diff, 0, lat, lon]
    return ref["fs"].V[t * time_diff, 0, lat, lon]

### today i learned

doing DataArray.values does not return a numpy array copy

In [ ]:
time_st = time.time()
target_interped_u = target["xrds"]["u"].values.copy()
target_interped_v = target["xrds"]["v"].values.copy()
invalid_interped = invalid.copy()
for f in interp_references:
    invalid_pos_new = np.where(invalid_interped)
    num_invalid_new = invalid_interped.sum()
    target_interped_u[invalid_pos_new] = [get_interped(i, f, "u", invalid_pos_new) for i in range(num_invalid_new)]
    target_interped_v[invalid_pos_new] = [get_interped(i, f, "v", invalid_pos_new) for i in range(num_invalid_new)]
    invalid_interped = generate_mask(target_interped_u)
time_en = time.time()
print(f"time elapsed: {time_en - time_st}")

In [ ]:
print(f"total invalid values on interpolated: {invalid_interped.sum()}")

### gridfill invalid values after interpolation

in theory there should still be invalid spaces left over because the 6 km data will sometimes have gaps

hopefully after this interpolation, the gaps of invalid data are small enough to let gridfilling finish the job

In [ ]:
import numpy.ma as ma
from gridfill import fill

masked_u = ma.masked_array(target_interped_u, invalid_interped)
masked_v = ma.masked_array(target_interped_v, invalid_interped)

kw = dict(eps=1e-4, relax=0.6, itermax=1e4, initzonal=False,
          cyclic=False, verbose=True)

# since data is 3d, use axes 2 and 1 since axis 0 is time
filled_u, converged_u = fill(masked_u, 2, 1, **kw)
filled_v, converged_v = fill(masked_v, 2, 1, **kw)

In [ ]:
invalid_filled = generate_mask(filled_u)
print(f"total invalid values on filled: {invalid_filled.sum()}")

In [ ]:
print(f"percent invalid filled: {(invalid_interped.sum() - invalid_filled.sum()) / invalid_interped.sum()}")

### wow

unsurprisingly gridfill did jack shit

### formatting, saving, and testing

In [ ]:
def conv_to_dataarray(arr, darr_ref):
    """
    Args:
        arr (array-like)
        darr_ref (xr.DataArray): only used to label coordinates, dimensions, and metadata.
    """
    return xr.DataArray(arr, coords=darr_ref.coords, dims=darr_ref.dims, attrs=darr_ref.attrs)

In [ ]:
# re-add coordinates, dimensions, and metadata to interpolated data
darr_u = conv_to_dataarray(filled_u, target["xrds"]["u"])
darr_v = conv_to_dataarray(filled_v, target["xrds"]["v"])
target_interped_xrds = target["xrds"].drop_vars(["u", "v"]).assign(u=darr_u, v=darr_v)
target_interped_xrds

In [ ]:
save_path = target["path"].split(".nc")[0] + "_interped.nc"
target_interped_xrds.to_netcdf(save_path)
print(f"saved to {save_path}")

### display field to see if interpolation worked

In [ ]:
fs_interp = FieldSet.from_xarray_dataset(target_interped_xrds.copy(deep=True), dict(U="u",V="v"), dict(lat="lat",lon="lon",time="time"))

In [ ]:
target["fs"].U.show()
fs_interp.U.show()